In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [3]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

df = pd.read_csv(root_folder + file_name)

In [4]:
X = df[['baro','imu_ax', 'imu_ay', 'imu_az','imu_gx', 'imu_gy', 'imu_gz','imu_mx', 'imu_my', 'imu_mz']].values
y = df['Texture']

print(f"Total samples collected: {X.shape[0]}")
print(f"Feature vector size: {X.shape[1]}")
print(f"Unique textures: {np.unique(y)}")

Total samples collected: 90665
Feature vector size: 10
Unique textures: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
texture_classes = label_encoder.classes_
num_classes = len(texture_classes)
print("\nEncoded texture labels:", texture_classes)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


Encoded texture labels: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [58]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)

class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [59]:
model = ANN(10, 100, 12)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [60]:
num_epochs = 20

for epoch in range(num_epochs):
    accuracy = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy += (y_batch == y_pred.argmax(1)).sum().item()
    print(f"Epoch {epoch+1}, loss: {loss.item()}, accuracy: {accuracy/X_train.shape[0]}")

Epoch 1, loss: 1.2311073541641235, accuracy: 0.5011581095240721
Epoch 2, loss: 1.1614422798156738, accuracy: 0.5809573705398996
Epoch 3, loss: 1.061217188835144, accuracy: 0.6159350355705068
Epoch 4, loss: 1.08589768409729, accuracy: 0.6365879887497932
Epoch 5, loss: 0.9966001510620117, accuracy: 0.6582336072354271
Epoch 6, loss: 0.7233521342277527, accuracy: 0.6695389621132741
Epoch 7, loss: 0.9301535487174988, accuracy: 0.6802101141565102
Epoch 8, loss: 0.9068951606750488, accuracy: 0.6940936414272321
Epoch 9, loss: 0.840896487236023, accuracy: 0.6960376109854961
Epoch 10, loss: 0.9592932462692261, accuracy: 0.704047868526995
Epoch 11, loss: 0.39922264218330383, accuracy: 0.7082115480063972
Epoch 12, loss: 0.6457757949829102, accuracy: 0.7125268846853803
Epoch 13, loss: 0.6245402097702026, accuracy: 0.7191032923399328
Epoch 14, loss: 0.8156502842903137, accuracy: 0.7208818176804721
Epoch 15, loss: 0.599629819393158, accuracy: 0.7219020570231071
Epoch 16, loss: 0.653846263885498, accu

In [53]:
# evaluation
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
with torch.no_grad():
    accuracy = 0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        accuracy += (y_batch == y_pred.argmax(1)).sum().item()
    print(f"Test accuracy: {accuracy/X_test.shape[0]}")

Test accuracy: 0.7484696409860475
